# Competitors

30 June, 2024

@roman

Code to find neighbors of a given competitor for each property.

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from tqdm import tqdm
import h3
import os

from scipy.spatial import KDTree
import scipy.sparse as sp
from scipy import io
from sklearn.cluster import MiniBatchKMeans, SpectralClustering, HDBSCAN
from INEGIpy import MarcoGeoestadistico

In [2]:
# Settings
# show 100 columns in pandas
pd.set_option('display.max_columns', 100)
geo_framework = MarcoGeoestadistico()

---
# Data

## Properties

In [3]:
# read parquet
gdf_properties = pd.read_parquet("../../data/interim/cleaned_data_s4.parquet")

# to geopandas
gdf_properties = gpd.GeoDataFrame(
    gdf_properties,
    geometry=gpd.points_from_xy(gdf_properties['longitud'], gdf_properties['latitud']),
    crs="EPSG:4326"
    )

# change crs to 6372
gdf_properties = gdf_properties.to_crs("EPSG:6372")

gdf_properties.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 852931 entries, 0 to 853032
Data columns (total 42 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   property_id                    852931 non-null  object        
 1   valor_fisico_construccion      852931 non-null  float64       
 2   id_avaluo                      852931 non-null  object        
 3   fecha_avaluo                   852931 non-null  datetime64[ns]
 4   colonia                        852931 non-null  object        
 5   id_clase_inmueble              852931 non-null  int64         
 6   id_tipo_inmueble               852931 non-null  int64         
 7   conservacion                   852931 non-null  float64       
 8   elevador                       824560 non-null  float64       
 9   niveles                        852931 non-null  int64         
 10  cve_ref_proximidad_urbana      852931 non-null  int64         
 1

## Cities

In [4]:
# read parquet
gdf_cities = gpd.read_parquet("../../data/misc/polygons_cities_analysis.parquet")

# change crs to 6372
gdf_cities = gdf_cities.to_crs("EPSG:6372")

gdf_cities.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   city_cluster  158 non-null    int64   
 1   geometry      158 non-null    geometry
 2   id_entidad_f  158 non-null    object  
 3   id_municipio  158 non-null    object  
dtypes: geometry(1), int64(1), object(2)
memory usage: 5.1+ KB


In [5]:
# buffer 1m around the cities to include the properties that are in the border
gdf_cities['geometry'] = gdf_cities.buffer(1)

In [6]:
# see table
gdf_cities.head()

,city_cluster,geometry,id_entidad_f,id_municipio
0,0,"POLYGON ((2802509.221 807490.857, 2802509.124 ...",09,003
1,1,"POLYGON ((1629955.411 1907388.013, 1629955.308...",26,030
2,2,"POLYGON ((3774584.759 1040368.498, 3774584.666...",31,050
3,3,"POLYGON ((1689809.905 1228250.719, 1689809.802...",03,008
4,4,"POLYGON ((2925744.611 1137556.825, 2925744.525...",28,003


## Wrangle

In [7]:
# wrangle
gdf_properties = (
    gdf_properties
    # new vars
    .assign(
        property_type=lambda x: np.where(x['id_tipo_inmueble'].le(3), 'house', 'apartment'),
        longitude=lambda x: x["geometry"].x,
        latitude=lambda x: x["geometry"].y,
        estacionamiento=lambda x: x["estacionamiento"].fillna(0),
        price_per_sqm=lambda x: x["valor_mercado"] / x["superficie_vendible"],
        log_price_per_sqm=lambda x: np.log(x["price_per_sqm"]),
    )
    .reset_index(drop=True)
    # categorize variables for comparisson
    .assign(
        recamaras_cat=lambda x: np.select(
            [
                x["recamaras"].le(1),
                x["recamaras"].le(2),
                x["recamaras"].le(3),
                x["recamaras"].gt(3),
            ],
            [
                1, 2, 3, 4
            ],
            default=0
        ),
        banos_cat=lambda x: np.select(
            [
                x["banos"].le(1),
                x["banos"].le(2),
                x["banos"].le(3),
                x["banos"].gt(3),
            ],
            [
                1, 2, 3, 4
            ],
            default=0
        ),
        niveles_cat=lambda x: 
            np.where(x['niveles'].le(1), 1, 2),
        vida_util_cat=lambda x: np.select(
            [
                x["vida_util_remanente"].le(30),
                x["vida_util_remanente"].le(60),
                x["vida_util_remanente"].le(70),
                x["vida_util_remanente"].gt(70),
            ],
            [
                1, 2, 3, 4
            ],
            default=0
        ),
        elevador_cat=lambda x: 
            np.where(x['elevador'].lt(1), 0, 1),
        estacionamiento_cat=lambda x: np.select(
            [
                x["estacionamiento"].le(0),
                x["estacionamiento"].le(1),
                x["estacionamiento"].le(2),
                x["estacionamiento"].gt(2),
            ],
            [
                0, 1, 2, 3
            ],
            default=0
        ),
    )
)   

In [8]:
# get area of each city
gdf_cities["area_city_km2"] = gdf_cities["geometry"].area / 1e+6
gdf_cities.head()

,city_cluster,geometry,id_entidad_f,id_municipio,area_city_km2
0,0,"POLYGON ((2802509.221 807490.857, 2802509.124 ...",09,003,2904.067758
1,1,"POLYGON ((1629955.411 1907388.013, 1629955.308...",26,030,209.597369
2,2,"POLYGON ((3774584.759 1040368.498, 3774584.666...",31,050,421.999965
3,3,"POLYGON ((1689809.905 1228250.719, 1689809.802...",03,008,51.848444
4,4,"POLYGON ((2925744.611 1137556.825, 2925744.525...",28,003,189.096787


In [9]:
# join data
gdf_properties = (
    gdf_properties
    .sjoin(
        gdf_cities.loc[:, ["city_cluster", "geometry", 'area_city_km2']],
        how="left",
    )
    .drop_duplicates(subset=["property_id"], keep="first")
    .drop(columns=["index_right"])
    .reset_index(drop=True)
)

# see shape
gdf_properties.shape

(852931, 55)

## Mini EDA

In [10]:
# see cat vars description
(
    gdf_properties
    .filter(like="_cat")
    .apply(pd.Series.value_counts)
)

,recamaras_cat,banos_cat,niveles_cat,vida_util_cat,elevador_cat,estacionamiento_cat
0,NaN,NaN,NaN,NaN,169201.0,104016.0
1,49383.0,574765.0,462058.0,14153.0,683730.0,566711.0
2,507910.0,199792.0,390873.0,565858.0,NaN,169035.0
3,262388.0,55115.0,NaN,232568.0,NaN,13169.0
4,33250.0,23259.0,NaN,40352.0,NaN,NaN


In [11]:
# see how many properties didnt join
gdf_properties["city_cluster"].isna().sum()

0

In [12]:
# if there is a index repeated
gdf_properties.index.duplicated().sum()

0

---
# Get Competitors

## S1: Calculate

In [13]:
# functions
def get_neighbors_properties(gdf, r=1):
    # copy
    gdf = gdf.copy()

    # fit kdtree
    kdtree = KDTree(
        data=gdf[['longitude', 'latitude']],
    )

    # get neighbors at r-km
    return kdtree.query_ball_point(
        gdf[['longitude', 'latitude']],
        r=r * 1_000,
        workers=-1
    )

def get_possible_neighbors(df_own, df_theirs, vars_list):
    return (
        df_own
        .loc[:, vars_list + ['neighbors_list']]
        .explode('neighbors_list')
        .rename(columns={'neighbors_list': 'id_neighbor'})
        .reset_index()
        .merge(
            (
                df_theirs
                .assign(index=lambda x: x.index)
                .loc[:, ['index'] + vars_list]
                ),
            how='inner',
            left_on='id_neighbor',
            right_on='index',
            suffixes=('_own', '_neighbor'),
        )
        .query("property_id_own != property_id_neighbor")  # remove self
    )


# get distance
def find_competitors(gdf):
    # Distance
    gdf_distance = (
        gdf.copy()
        .assign(
            # geo distance
            geo_distance=lambda x: np.sqrt(np.sqrt(
                (x['longitude_own'] - x['longitude_neighbor'])**2
                + (x['latitude_own'] - x['latitude_neighbor'])**2
            ) / 1e+3),  # normalize to 0-1
            # topology distance
            terrain_distance=lambda x: np.abs(1-x['superficie_terreno_neighbor'] / x['superficie_terreno_own']).clip(0, 1),
            built_distance=lambda x: np.sqrt(np.abs(1-x['superficie_construida_neighbor'] / x['superficie_construida_own']).clip(0, 1)),
            # characteristics distance
            characteristics_distance=lambda x: np.sqrt(np.sqrt(
                (x['elevador_cat_own'] - x['elevador_cat_neighbor'])**2  # 0s & 1s
                + (x['niveles_cat_own'] - x['niveles_cat_neighbor'])**2  # 1s & 2s (idem)
                + (x['vida_util_cat_own'] - x['vida_util_cat_neighbor'])**2  
                + (x['recamaras_cat_own'] - x['recamaras_cat_neighbor'])**2
                + (x['banos_cat_own'] - x['banos_cat_neighbor'])**2
                + (x['estacionamiento_cat_own'] - x['estacionamiento_cat_neighbor'])**2
            ) / (0.5 * np.sqrt(1 + 1 + 3**2 + 3**2 + 3**2 + 3**2))),
            # time distance (relu)
            time_distance_raw=lambda x: (
               (x['fecha_avaluo_own'] - x['fecha_avaluo_neighbor']).dt.days
            ),
            time_distance=lambda x: np.square(x['time_distance_raw'] / (365 * 2)).clip(0, 1)
        )
        # filter out those that are more than a year apart
        .query("time_distance.le(1)")
        .assign(
            # total distance
            total_distance=lambda x: (
                x['geo_distance'] * GEO_WEIGHT
                + x['terrain_distance'] * TERRAIN_WEIGHT
                + x['built_distance'] * BUILT_WEIGHT
                + x['characteristics_distance'] * CHAR_WEIGHT
                + x['time_distance'] * TIME_WEIGHT
            )
        )
        # filter all those that are above 1 of distance
        # .query("total_distance.lt(1)")
        .query("total_distance.gt(0)")  # eliminate 0s in the distance to avoid comparing to itself
    )
    # Aggregation
    df_comps = (
        gdf_distance
        .assign(
            index_distance_tuple=lambda x: list(zip(x['property_id_neighbor'], x['total_distance'])),
        )
        .groupby('property_id_own', as_index=False)
        .agg(
            neighbors_list=('index_distance_tuple', list),
            num_neighbors=('property_id_neighbor', 'count'),
        )
    )
    return df_comps


# function to orchestrate the process
def get_competitors(gdf, property_type='apartment', city=0, batch_size=None, radius=1):
    # params
    cols_to_stay = [
        # variables to stay
        'property_id',
        # geographic
        'longitude', 'latitude',
        # topology
        'superficie_terreno', 'superficie_construida',
        # characteristics
        'elevador_cat', 'niveles_cat', 'vida_util_cat', 
        'recamaras_cat', 'banos_cat', 'estacionamiento_cat',
        # time
        'fecha_avaluo',
    ]

    # subset 
    gdf_work = (
        gdf
        .query("city_cluster.eq(@city) & property_type.eq(@property_type)")
        .copy()
    )

    # check if there are properties
    if gdf_work.shape[0] == 0:
        return pd.DataFrame()

    # get neighbors
    gdf_work['neighbors_list'] = get_neighbors_properties(gdf_work, r=radius)

    # get possible neighbors by batch
    n_batches = gdf_work.shape[0] // batch_size
    if n_batches > 0:
        batch_indexes = np.array_split(gdf_work.index, gdf_work.shape[0] // batch_size)
    else:
        batch_indexes = [gdf_work.index]

    gdf_neighbors_list = []
    for batch in batch_indexes:
        # print batch
        # get possible neighbors
        gdf_neighbors_info = get_possible_neighbors(
            df_own=gdf_work.loc[batch],
            df_theirs=gdf_work,
            vars_list=cols_to_stay
        )
        # find competitors
        gdf_neighbors_subset = find_competitors(gdf_neighbors_info)
        # append
        gdf_neighbors_list.append(gdf_neighbors_subset)

    # concatenate
    gdf_neighbors = pd.concat(gdf_neighbors_list)
    
    return gdf_neighbors

In [14]:
# Get competitors
# city values
cities_list = gdf_properties['city_cluster'].sort_values().unique()
property_types_list = gdf_properties['property_type'].unique()

# params
GEO_WEIGHT = 0.1
TERRAIN_WEIGHT = 0.1
BUILT_WEIGHT = 0.3
CHAR_WEIGHT = 0.3
TIME_WEIGHT = 0.2

# calculate
df_competitors_list = []
for city in tqdm(cities_list, desc="Cities", position=0):
    for property_type in property_types_list:
        # get competitors
        df_comps = get_competitors(
            gdf_properties,
            property_type=property_type,
            city=city,
            batch_size=2_000,
            radius=2.5  # search radius in km
            )
        # append
        df_competitors_list.append(df_comps)
    
# concatenate
df_competitors = pd.concat(df_competitors_list)

Cities:   0%|          | 0/158 [03:35<?, ?it/s]


KeyboardInterrupt: 

## S2: Explore

In [ ]:
# see shape
df_competitors['num_neighbors'].describe()

In [ ]:
# see shape
df_competitors.shape

In [ ]:
# see shape vs original
gdf_properties.query("city_cluster.eq(0)").shape

In [ ]:
# see competitors with only 1
df_competitors.sort_values(by='num_neighbors', ascending=False)

In [ ]:
# expand and separate
vars_to_stay = [
    'property_id', 'property_type',
    'longitude', 'latitude',
    'superficie_terreno', 'superficie_construida',
    'elevador_cat', 'niveles_cat', 'vida_util_cat',
    'recamaras_cat', 'banos_cat', 'estacionamiento_cat',
    'fecha_avaluo', 'valor_mercado', 'precio_m2'
]

(
    df_competitors
    .query("property_id_own.eq('5cc32435a1224836af921c93f0d8cb2a')")
    .explode('neighbors_list')
    .assign(
        property_id_neighbor=lambda x: x['neighbors_list'].str[0],
        total_distance=lambda x: x['neighbors_list'].str[1],
    )
    .drop(columns=['neighbors_list'])
    .sort_values(by='total_distance', ignore_index=True)
    # merge info of property_own
    .merge(
        gdf_properties.loc[:, vars_to_stay],
        how='inner',
        left_on='property_id_own',
        right_on='property_id',
        suffixes=('_own', '_neighbor'),
    )
    .drop(columns=['property_id'])
    # merge info of property_neighbor
    .merge(
        gdf_properties.loc[:, vars_to_stay],
        how='inner',
        left_on='property_id_neighbor',
        right_on='property_id',
        suffixes=('_own', '_neighbor'),
    )
    # sort columns by name
    .sort_index(axis=1)
)

## S3: Create Sparse Matrix

In [ ]:
# create df that maps property_id with index
df_properties_map = (
    gdf_properties
    .loc[:, ['property_id']]
    .reset_index()
    .rename(columns={'index': 'property_index'})
)
df_properties_map

In [ ]:
# create function to map property_id to index
def create_sparse_matrix_in_batches(df, df_indexes, batch_size=1_000):
    """
    Convert a large DataFrame with property IDs and neighbor distances into a sparse matrix in batches.

    Parameters:
    df (pd.DataFrame): DataFrame with 'property_id_own' and 'neighbors_list' columns.
                       'property_id_own' contains item IDs.
                       'neighbors_list' contains lists of tuples (neighbor_id, distance).
    df_indexes (pd.DataFrame): DataFrame with 'property_id' and 'property_index' columns.
                                 'property_id' contains item IDs.
                                    'property_index' contains item indices.
    batch_size (int): The number of rows to process in each batch.

    Returns:
    scipy.sparse.csr_matrix: The resulting sparse matrix.
    """
    # Split DataFrame into approximately equal-sized batches
    num_batches = df.shape[0] // batch_size
    chunks = np.array_split(df, num_batches)

    # Initialize lists
    # info
    row_list = []
    col_list = []
    data_list = []

    # garantize the sparse matrix is square
    max_id = df_indexes['property_index'].max()

    for chunk in tqdm(chunks, desc="Batches"):
        # Expand chunk
        exploded_chunk = chunk.explode('neighbors_list').reset_index(drop=True)

        # Merge with index map
        exploded_chunk_ids = (
            exploded_chunk
            # property id
            .merge(df_indexes, left_on='property_id_own', right_on='property_id')
            .loc[:, ['property_index', 'neighbors_list']]
            # neighbor id
            .assign(
                property_id_neighbor=lambda x: x['neighbors_list'].str[0],
                total_distance=lambda x: x['neighbors_list'].str[1]
            )
            .merge(df_indexes, left_on='property_id_neighbor', right_on='property_id', suffixes=('_own', '_neighbor'))
            .loc[:, ['property_index_own', 'property_index_neighbor', 'total_distance']]
        )

        # Extract row indices, column indices, and data values
        rows = exploded_chunk_ids['property_index_own'].values
        cols = exploded_chunk_ids['property_index_neighbor'].values
        distances = exploded_chunk_ids['total_distance'].values

        # Append to lists
        row_list.append(rows)
        col_list.append(cols)
        data_list.append(distances)

    # Concatenate all chunks
    rows = np.concatenate(row_list)
    cols = np.concatenate(col_list)
    distances = np.concatenate(data_list)

    # Create the sparse matrix, garanting it is square
    sparse_matrix = sp.csr_matrix((distances, (rows, cols)), shape=(max_id + 1, max_id + 1))

    return sparse_matrix

# Example usage
sparse_matrix_comps = create_sparse_matrix_in_batches(
    df=df_competitors,
    df_indexes=df_properties_map,
    batch_size=10_000
    )

print(sparse_matrix_comps.shape)

In [ ]:
# see shape
sparse_matrix_comps.shape

In [ ]:
# see how many non-zero elements
print(f"Non-zero elements: {sparse_matrix_comps.nnz}")
print(f"Sparsity: {sparse_matrix_comps.nnz / (sparse_matrix_comps.shape[0] * sparse_matrix_comps.shape[1])}")

In [ ]:
# how much memory it uses in MB
print(f"Memory usage: {sparse_matrix_comps.data.nbytes / 1e+6:.2f} MB")

## S4: Save

In [ ]:
# save
dir_save = "../../data/misc"
os.makedirs(dir_save, exist_ok=True)

# save to pickle (in order to save lists, because parquet doesnt support lists)
df_competitors.to_pickle(f"{dir_save}/df_competitors.pkl")

In [ ]:
# save sparse matrix
sp.save_npz(f"{dir_save}/sparse_matrix_competitors.npz", sparse_matrix_comps)

In [ ]:
# save in matlab format
io.savemat(f"{dir_save}/sparse_matrix_competitors.mat", {"sparse_matrix_competitors": sparse_matrix_comps})

---
# Competitors Statistics

## S1: Define statistics

In [ ]:
# summary statistics of competitors
def competitors_stats(df, col_to_summarize, xi=0.2):
    # get normalized weights
    weights_unorm = np.exp(-xi * df['total_distance'])
    weights = weights_unorm / weights_unorm.sum()
    n_neighbors = df.shape[0]
    sq_sum_weights = np.sum(weights**2) if n_neighbors > 1 else np.nan

    # get col to summarize
    x_array = df[col_to_summarize].values

    # weighted mean
    w_mean = np.sum(x_array * weights)

    return pd.Series({
        'weighted_mean': w_mean,
        'weighted_std': np.sqrt(
            # unbiased weighted std (theorem)
            (1 / (1 - sq_sum_weights)) * np.sum(weights * (x_array - w_mean)**2)
            ),
        'mean': np.mean(x_array),
        'std': np.std(x_array),
        'num_neighbors': n_neighbors,
    })


def get_info_from_competitors(df, df_info, col_to_summarize, xi=0.2, batch_size=2_000):
    # Split DataFrame into approximately equal-sized batches
    num_batches = df.shape[0] // batch_size
    chunks = np.array_split(df, num_batches)

    # Initialize list
    df_competitors_info_list = []

    for chunk in tqdm(chunks, desc="Batches"):
        # Expand chunk
        exploded_chunk = chunk.explode('neighbors_list').reset_index(drop=True)

        # Merge with info
        exploded_chunk = (
            exploded_chunk
            # neighbor id
            .assign(
                property_id_neighbor=lambda x: x['neighbors_list'].str[0],
                total_distance=lambda x: x['neighbors_list'].str[1]
            )
            .merge(
                df_info.loc[:, ['property_id'] + [col_to_summarize]],
                left_on='property_id_neighbor', right_on='property_id',
                suffixes=('_own', '_neighbor')
            )
            .drop(columns=['property_id', 'neighbors_list'])
        )

        # get stats
        df_competitors_info = (
            exploded_chunk
            .groupby('property_id_own', as_index=False)
            .apply(
                competitors_stats,
                col_to_summarize=col_to_summarize,
                xi=xi,
                include_groups=False
                )
        )

        # append
        df_competitors_info_list.append(df_competitors_info)

    # concatenate
    df_competitors_info = pd.concat(df_competitors_info_list)

    return df_competitors_info

In [ ]:
# get info from competitors
df_competitors_info = get_info_from_competitors(
    df=df_competitors,
    df_info=gdf_properties,
    col_to_summarize='log_price_per_sqm',
    xi=5,
    batch_size=10_000
    )
df_competitors_info

In [ ]:
# rename
df_competitors_info = df_competitors_info.rename(columns={
    'weighted_mean': 'competitors_weighted_mean_log_price_per_sqm',
    'weighted_std': 'competitors_weighted_std_log_price_per_sqm',
    'mean': 'competitors_mean_log_price_per_sqm',
    'std': 'competitors_std_log_price_per_sqm',
    'num_neighbors': 'num_competitors',
})
df_competitors_info.head()

## S2: EDA

In [ ]:
# merge with original data
gdf_properties = (
    gdf_properties
    .merge(
        df_competitors_info,
        how='left',
        left_on='property_id',
        right_on='property_id_own',
    )
    .drop(columns=['property_id_own'])
)

# shape
gdf_properties.shape

In [ ]:
# see how many properties have competitors
gdf_properties['num_competitors'].describe()

In [ ]:
# see relationship between log_price_per_sqm and competitors_weighted_mean_log_price_per_sqm
fig, ax = plt.subplots(figsize=(6, 6))

(
    gdf_properties
    .query("num_competitors.ge(4)")
    .plot(
        x='log_price_per_sqm',
        y='competitors_weighted_mean_log_price_per_sqm',
        kind='scatter',
        ax=ax,
        alpha=0.1
    )
)

In [ ]:
# see relationship between log_price_per_sqm and competitors_weighted_mean_log_price_per_sqm
fig, ax = plt.subplots(figsize=(6, 6))

(
    gdf_properties
    .query("num_competitors.ge(4)")
    .plot(
        x='log_price_per_sqm',
        y='competitors_mean_log_price_per_sqm',
        kind='scatter',
        ax=ax,
        alpha=0.1
    )
)

In [ ]:
# see corr between log_price_per_sqm and competitors_weighted_mean_log_price_per_sqm and competitors_mean_log_price_per_sqm
(
    gdf_properties
    .query("num_competitors.ge(4)")
    .loc[:, ['log_price_per_sqm', 'competitors_weighted_mean_log_price_per_sqm', 'competitors_mean_log_price_per_sqm']]
    .corr(method='spearman')
)

## S3: Save

In [ ]:
# rename
df_competitors_info = df_competitors_info.rename(columns={
    'property_id_own': 'property_id'
})
df_competitors_info.head()

In [ ]:
# save
dir_save = "../../data/misc"
os.makedirs(dir_save, exist_ok=True)

# save to parquet
df_competitors_info.to_parquet(f"{dir_save}/df_competitors_stats.parquet")

---
# Save

In [ ]:
df_competitors_info.drop(columns='property_id').columns

In [ ]:
# drop competitors columns
gdf_properties.drop(columns=df_competitors_info.drop(columns='property_id').columns, inplace=True)

# sort index
gdf_properties = gdf_properties.sort_index()

# save to parquet
gdf_properties.to_parquet("../../data/interim/cleaned_data_s6.parquet")

In [ ]:
# stopper
fjasdklf + cdsajofasd

---
# Sandbox

In [ ]:
(gdf_neighbors['fecha_avaluo_own'] - gdf_neighbors['fecha_avaluo_neighbor']).dt.days

In [ ]:
gdf_neighbors.columns

In [ ]:
# join with df_competitors
(
    df_competitors
    .merge(
        df_properties_map,
        how='inner',
        left_on='property_id_own',
        right_on='property_id',
        suffixes=('_comp', '_map')
    )
)

In [ ]:
df_competitors

In [ ]:
gdf_properties.shape

In [ ]:
np.square(3)

In [ ]:
gdf_properties_work['property_id']

In [ ]:
(1.96)**2

In [ ]:
np.array_split(np.arange(10), 4)

In [ ]:
aux_list = []

# append
aux_list.append(2)

aux_list

In [ ]:
gdf_properties.columns

In [ ]:
?np.array_split